# Forecasting: Stock Prediction 📊

Datasets source: Yahoo finance [Yfinance python library](https://pypi.org/project/yfinance/)

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import plotly.express as px
import plotly.io as pio
import dask
import warnings
import yfinance as yf
import plotly.graph_objects as go
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Flatten
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from datetime import date
from datetime import timedelta
import joblib
today = date.today()
print("Today is: ", today)

yesterday = today - timedelta(days = 1)
print("Yesterday was: ", yesterday)

warnings.filterwarnings('ignore')
pio.renderers.default = 'vscode'

px.defaults.template = "plotly_dark"

Today is:  2024-12-27
Yesterday was:  2024-12-26


## Download datasets from Yahoo Finannce

In [2]:
df_apple = yf.download('AAPL', start='2020-01-01', end=yesterday)
df_samsung = yf.download('005930.KS', start='2020-01-01', end=yesterday)
df_xiaomi = yf.download('1810.HK', start='2020-01-01', end=yesterday)
df_nvidia = yf.download('NVDA', start='2020-01-01', end=yesterday)
df_google = yf.download('GOOG', start='2020-01-01', end=yesterday)
df_amazon = yf.download('AMZN', start='2020-01-01', end=yesterday)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [3]:
def edit_df(df):
    """change column's name and reset index"""
    df.columns = [c[0] for c in df.columns]
    df = df.reset_index()
    return df

In [4]:
df_apple = edit_df(df_apple)
df_samsung = edit_df(df_samsung)
df_xiaomi = edit_df(df_xiaomi)
df_nvidia = edit_df(df_nvidia)
df_google = edit_df(df_google)
df_amazon = edit_df(df_amazon)

In [5]:
df_apple

,Date,Close,High,Low,Open,Volume
0,2020-01-02,72.796028,72.856621,71.545395,71.799881,135480400
1,2020-01-03,72.088280,72.851746,71.862877,72.020416,146322800
2,2020-01-06,72.662720,72.701500,70.954010,71.206077,118387200
3,2020-01-07,72.320953,72.929298,72.100395,72.672386,108872000
4,2020-01-08,73.484344,73.787308,72.022850,72.022850,132079200
...,...,...,...,...,...,...
1249,2024-12-18,248.050003,254.279999,247.740005,252.160004,56774100
1250,2024-12-19,249.789993,252.000000,247.089996,247.500000,60882300
1251,2024-12-20,254.490005,255.000000,245.690002,248.039993,147495300
1252,2024-12-23,255.270004,255.649994,253.449997,254.770004,40858800


In [6]:
df_samsung

,Date,Close,High,Low,Open,Volume
0,2020-01-02,48825.468750,49533.084239,48648.564878,49090.824558,12993228
1,2020-01-03,49090.828125,50063.799493,48560.116470,49533.087838,15422255
2,2020-01-06,49090.828125,49179.280068,48294.760642,48560.116470,10278951
3,2020-01-07,49356.183594,49886.895245,49179.279710,49267.731652,10009778
4,2020-01-08,50240.695312,50771.406883,49444.627957,49709.983742,23501171
...,...,...,...,...,...,...
1221,2024-12-18,54900.000000,55400.000000,54000.000000,54100.000000,13698937
1222,2024-12-19,53100.000000,53800.000000,53100.000000,53500.000000,22481925
1223,2024-12-20,53000.000000,53100.000000,51900.000000,52700.000000,24674774
1224,2024-12-23,53500.000000,54000.000000,53300.000000,53400.000000,13672650


In [7]:
df_xiaomi

,Date,Close,High,Low,Open,Volume
0,2020-01-02,11.220000,11.220000,10.780000,10.860000,248889455
1,2020-01-03,10.900000,11.440000,10.900000,11.200000,219547199
2,2020-01-06,10.980000,11.180000,10.640000,10.720000,179639996
3,2020-01-07,11.000000,11.300000,10.920000,11.060000,171287427
4,2020-01-08,11.080000,11.280000,10.820000,10.820000,189314498
...,...,...,...,...,...,...
1222,2024-12-18,30.799999,30.900000,30.049999,30.450001,86680411
1223,2024-12-19,30.799999,31.049999,30.000000,30.200001,85886142
1224,2024-12-20,31.650000,32.099998,30.700001,30.700001,204413958
1225,2024-12-23,31.750000,32.250000,31.450001,32.000000,97720054


In [8]:
df_nvidia

,Date,Close,High,Low,Open,Volume
0,2020-01-02,5.972710,5.972710,5.893293,5.943831,237536000
1,2020-01-03,5.877110,5.920927,5.828066,5.852962,205384000
2,2020-01-06,5.901758,5.906986,5.757613,5.783753,262636000
3,2020-01-07,5.973208,6.019016,5.885078,5.930139,314856000
4,2020-01-08,5.984411,6.025738,5.928894,5.968976,277108000
...,...,...,...,...,...,...
1249,2024-12-18,128.910004,136.699997,128.279999,133.860001,277444500
1250,2024-12-19,130.679993,134.029999,129.550003,131.759995,209719200
1251,2024-12-20,134.699997,135.279999,128.220001,129.809998,306528600
1252,2024-12-23,139.669998,139.789993,135.119995,136.279999,176053500


In [9]:
df_google

,Date,Close,High,Low,Open,Volume
0,2020-01-02,68.123726,68.162086,66.837348,66.837348,28132000
1,2020-01-03,67.789421,68.379304,67.036329,67.151713,23728000
2,2020-01-06,69.460922,69.575007,67.258334,67.258334,34646000
3,2020-01-07,69.417580,69.898350,69.270107,69.646760,30054000
4,2020-01-08,69.964615,70.326314,69.293024,69.354799,30560000
...,...,...,...,...,...,...
1249,2024-12-18,190.149994,198.690002,189.279999,196.830002,27638400
1250,2024-12-19,189.699997,194.600006,189.520004,193.279999,26981200
1251,2024-12-20,192.960007,194.134995,186.369995,187.009995,45319700
1252,2024-12-23,195.990005,196.490005,191.630005,194.029999,15235900


In [10]:
df_amazon

,Date,Close,High,Low,Open,Volume
0,2020-01-02,94.900497,94.900497,93.207497,93.750000,80580000
1,2020-01-03,93.748497,94.309998,93.224998,93.224998,75288000
2,2020-01-06,95.143997,95.184502,93.000000,93.000000,81236000
3,2020-01-07,95.343002,95.694504,94.601997,95.224998,80898000
4,2020-01-08,94.598503,95.550003,94.321999,94.902000,70160000
...,...,...,...,...,...,...
1249,2024-12-18,220.520004,231.399994,220.110001,230.770004,43281400
1250,2024-12-19,223.289993,226.089996,222.919998,224.910004,39918700
1251,2024-12-20,224.919998,226.210007,218.729996,219.839996,88279200
1252,2024-12-23,225.059998,226.880005,223.899994,225.009995,28070000


### Datasets are made of the following:
- **Open:** Opening stock price of the day
- **Close:** Closing stock price of the day
- **High:** Highest stock price of the data
- **Low:** Lowest stock price of the day
- **Volume:** total number of shares traded during a specific period

## Data visualisation: lines and candlestick graphs

In [11]:
def lines_graph(df, sym):
    """lines graphs"""

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df['Date'],
                             y=df['Close'],
                             mode='lines',
                             name='Close Price'))

    fig.update_layout(
        title=f"Closing Price of {((yf.Ticker(sym)).info)['longName']} Stock",
        xaxis_title="Date",
        yaxis_title="Close Price ($)",
        height=500,
        margin=dict(t=50, b=50)
    )
    fig.show()

In [12]:
lines_graph(df_apple, 'AAPL')
lines_graph(df_samsung, '005930.KS')
lines_graph(df_xiaomi, '1810.HK')
lines_graph(df_nvidia, 'NVDA')
lines_graph(df_google, 'GOOG')
lines_graph(df_amazon, 'AMZN')


In [13]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_apple['Date'],
                         y=df_apple['Close'].interpolate(),
                         mode='lines',
                         name='Apple Close Price'))
fig.add_trace(go.Scatter(x=df_samsung['Date'],
                         y=df_samsung['Close'].interpolate(),
                         mode='lines',
                         name='SamsungClose Price'))
fig.add_trace(go.Scatter(x=df_xiaomi['Date'],
                         y=df_xiaomi['Close'].interpolate(),
                         mode='lines',
                         name='Xiaomi Close Price'))
fig.add_trace(go.Scatter(x=df_nvidia['Date'],
                         y=df_nvidia['Close'].interpolate(),
                         mode='lines',
                         name='Nvidia Close Price'))
fig.add_trace(go.Scatter(x=df_google['Date'],
                            y=df_google['Close'].interpolate(),
                            mode='lines',
                            name='Google Close Price'))
fig.add_trace(go.Scatter(x=df_amazon['Date'],
                            y=df_amazon['Close'].interpolate(),
                            mode='lines',
                            name='Amazon Close Price'))


fig.update_layout(
    title="Closing Price of Apple, Xiaomi, Samsung, Nvidia, Google, and Amazon Stock",
    xaxis_title="Date",
    yaxis_title="Close Price ($)",
    height=500,
    margin=dict(t=50, b=50))


fig.show()


In [14]:
def candlestick_graph(df, sym):
    """candlestick graph"""
    fig = go.Figure(data=go.Candlestick(x=df['Date'], open=df[f'Open'],  high=df[f'High'],
                    low=df[f'Low'],
                    close=df[f'Close']))
    fig.update_layout(
        title=f"{((yf.Ticker(sym)).info)['longName']} stock candlestick chart ($)",
        height=500,
        margin=dict(t=50, b=50)
    )

    fig.show()

In [15]:
candlestick_graph(df_apple, "AAPL")
candlestick_graph(df_samsung, '005930.KS')
candlestick_graph(df_xiaomi, '1810.HK')
candlestick_graph(df_nvidia, 'NVDA')
candlestick_graph(df_google, 'GOOG')
candlestick_graph(df_amazon, 'AMZN')


## Train-Test split

We split the datasets as following:
- **10%** for test 
- **90%** for train

In [16]:
# split datasets to train and test
def split_data(df):
    """split datasets to train and test"""
    train_size = int(len(df) * 0.9)
    test_size = len(df) - train_size
    train, test = df.iloc[0:train_size], df.iloc[train_size:len(df)]
    return train, test

train_apple, test_apple = split_data(df_apple)
train_samsung, test_samsung = split_data(df_samsung)
train_xiaomi, test_xiaomi = split_data(df_xiaomi)
train_nvidia, test_nvidia = split_data(df_nvidia)
train_google, test_google = split_data(df_google)
train_amazon, test_amazon = split_data(df_amazon)


In [17]:
def train_test_graph(df, sym, train_data, test_data):
    """train and test graph"""
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=train_data['Date'],
                            y=train_data['Close'],
                            mode='lines',
                            name='Train close Price'))
    fig.add_trace(go.Scatter(x=test_data['Date'],
                            y=test_data['Close'],
                            mode='lines',
                            name='Test close Price'))

    fig.update_layout(
        title=f"Closing Price of {((yf.Ticker(sym)).info)['longName']} Stock ($)",
        xaxis_title="Date",
        yaxis_title="Close Price ($)",
        height=500,
        margin=dict(t=50, b=50)
    )
    fig.show()

train_test_graph(df_apple, 'AAPL', train_apple, test_apple)
train_test_graph(df_samsung, '005930.KS', train_samsung, test_samsung)
train_test_graph(df_xiaomi, '1810.HK', train_xiaomi, test_xiaomi)
train_test_graph(df_nvidia, 'NVDA', train_nvidia, test_nvidia)
train_test_graph(df_google, 'GOOG', train_google, test_google)
train_test_graph(df_amazon, 'AMZN', train_amazon, test_amazon)

## Normalization: MinMax Scaling

In [19]:
def normalize_data(train, test):
    """normalize data"""
    scaler = MinMaxScaler()
    train = scaler.fit_transform(np.array(train['Close']).reshape(-1, 1))
    test = scaler.transform(np.array(test['Close']).reshape(-1, 1))
    return train, test, scaler

normalized_train_apple, normalized_test_apple, scaler_apple = normalize_data(train_apple, test_apple)
normalized_train_samsung, normalized_test_samsung, scaler_samsung = normalize_data(train_samsung, test_samsung)
normalized_train_xiaomi, normalized_test_xiaomi, scaler_xiaomi = normalize_data(train_xiaomi, test_xiaomi)
normalized_train_nvidia, normalized_test_nvidia, scaler_nvidia = normalize_data(train_nvidia, test_nvidia)
normalized_train_google, normalized_test_google, scaler_google = normalize_data(train_google, test_google)
normalized_train_amazon, normalized_test_amazon, scaler_amazon = normalize_data(train_amazon, test_amazon)


In [21]:
def evaluation_model(normalized_test, y_pred, history, modelname):
    """evaluat the model"""
    loss_df = pd.DataFrame({
        'Epochs': range(1, len(history.history['loss']) + 1),
        'Training Loss': history.history['loss'],
        'Validation Loss': history.history['val_loss']
    })

    fig = px.line(loss_df, x='Epochs', y=['Training Loss', 'Validation Loss'],
                  labels={'value': 'Loss', 'variable': 'Loss Type'},
                  title='Training and Validation Loss')

    fig.update_layout(legend_title_text='Loss Type')

    fig.show()
    normalized_test = np.array(normalized_test).flatten()
    y_pred = np.array(y_pred).flatten()

    mae = mean_absolute_error(normalized_test, y_pred)
    mse = mean_squared_error(normalized_test, y_pred)
    r2 = r2_score(normalized_test, y_pred)
    rmse = np.sqrt(mean_squared_error(normalized_test, y_pred))
    print(f"Root Mean Squared Error: {rmse}")
    print(f"Mean Absolute Error: {mae}")
    print(f"Mean Squared Error: {mse}")
    print(f"R2 Score: {r2}")

    trueval_df = pd.DataFrame({
        'Index': range(len(normalized_test)),
        'True Values': normalized_test,
        'Predictions': y_pred
    })

    fig = px.line(trueval_df, x='Index', y=['True Values', 'Predictions'],title=f'{modelname} Predictions vs True Values')

    fig.add_scatter(x=trueval_df.index, y=trueval_df['True Values'], mode='lines', name='True Values')
    fig.add_scatter(x=trueval_df.index, y=trueval_df['Predictions'], mode='lines', name='Predictions')

    fig.show()


## Build a neural network

**LSTM architectures (Long Short Term Memory)** are capable of learning long-term dependencies in sequential data, which makes them well-suited for **time series forecasting**.

In [22]:
np.random.seed(1234)
tf.random.set_seed(1234)
def LSTM_model(train, test, modelname, scaler):
    """Create and train the LSTM model"""

    model = Sequential([
        LSTM(64, return_sequences=True, input_shape=(train.shape[1], 1)),
        Dropout(0.2),
        LSTM(32, return_sequences=True),
        Dropout(0.2),
        LSTM(16),
        Dropout(0.2),
        Dense(1)
    ])
    model.compile(loss='mean_squared_error', optimizer='adam')
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min')
    model_checkpoint = ModelCheckpoint(f'{modelname}.keras', monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    history = model.fit(
        train, train,
        epochs=50,
        batch_size=32,
        validation_split=0.1,
        verbose=1,
        callbacks=[early_stopping, model_checkpoint]
    )

    # Predictions
    y_pred = model.predict(test)
    evaluation_model(test, y_pred, history, modelname)
    model.save(f"webapp/models/{modelname}_model.keras")
    joblib.dump(scaler, f"webapp/models/scaler_{modelname}.pkl")


In [30]:
LSTM_model(
   normalized_train_apple,
   normalized_test_apple,
    'apple',
    scaler_apple
)

Epoch 1/50
30/32 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.2961
Epoch 1: val_loss improved from inf to 0.43590, saving model to apple.keras
32/32 ━━━━━━━━━━━━━━━━━━━━ 51s 341ms/step - loss: 0.2934 - val_loss: 0.4359
Epoch 2/50
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.1537
Epoch 2: val_loss improved from 0.43590 to 0.02593, saving model to apple.keras
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 53ms/step - loss: 0.1507 - val_loss: 0.0259
Epoch 3/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.0299
Epoch 3: val_loss did not improve from 0.02593
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 64ms/step - loss: 0.0298 - val_loss: 0.0327
Epoch 4/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - loss: 0.0234
Epoch 4: val_loss improved from 0.02593 to 0.01713, saving model to apple.keras
32/32 ━━━━━━━━━━━━━━━━━━━━ 4s 129ms/step - loss: 0.0233 - val_loss: 0.0171
Epoch 5/50
30/32 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0166
Epoch 5: val_loss improved from 0.01713 to 0.00657, saving model to apple.keras
32/32 ━━

Root Mean Squared Error: 0.04149572169821312
Mean Absolute Error: 0.03970947521540146
Mean Squared Error: 0.0017218949192555548
R2 Score: 0.540781591993141


Training and validation loss plot shows good convergence, with both losses decreasing and stabilizing, indicating no major overfitting or underfitting issues.


In [24]:
LSTM_model(
   normalized_train_samsung,
   normalized_test_samsung,
    'samsung',
    scaler_samsung
)

Epoch 1/50
30/31 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.3224
Epoch 1: val_loss improved from inf to 0.49047, saving model to samsung.keras
31/31 ━━━━━━━━━━━━━━━━━━━━ 32s 189ms/step - loss: 0.3203 - val_loss: 0.4905
Epoch 2/50
30/31 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.1894
Epoch 2: val_loss improved from 0.49047 to 0.05709, saving model to samsung.keras
31/31 ━━━━━━━━━━━━━━━━━━━━ 6s 32ms/step - loss: 0.1858 - val_loss: 0.0571
Epoch 3/50
29/31 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0278
Epoch 3: val_loss improved from 0.05709 to 0.04069, saving model to samsung.keras
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 0.0278 - val_loss: 0.0407
Epoch 4/50
29/31 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0240
Epoch 4: val_loss improved from 0.04069 to 0.02169, saving model to samsung.keras
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 0.0238 - val_loss: 0.0217
Epoch 5/50
28/31 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0189
Epoch 5: val_loss improved from 0.02169 to 0.011

Root Mean Squared Error: 0.0239821917976613
Mean Absolute Error: 0.02087043322713074
Mean Squared Error: 0.000575145523419813
R2 Score: 0.9904642008812486


In [25]:
LSTM_model(
   normalized_train_xiaomi,
   normalized_test_xiaomi,
    'xiaomi',
    scaler_xiaomi
)

Epoch 1/50
30/32 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.1278
Epoch 1: val_loss improved from inf to 0.03202, saving model to xiaomi.keras
32/32 ━━━━━━━━━━━━━━━━━━━━ 35s 172ms/step - loss: 0.1275 - val_loss: 0.0320
Epoch 2/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0661
Epoch 2: val_loss improved from 0.03202 to 0.01160, saving model to xiaomi.keras
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 49ms/step - loss: 0.0658 - val_loss: 0.0116
Epoch 3/50
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.0347
Epoch 3: val_loss improved from 0.01160 to 0.00586, saving model to xiaomi.keras
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step - loss: 0.0345 - val_loss: 0.0059
Epoch 4/50
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.0205
Epoch 4: val_loss improved from 0.00586 to 0.00274, saving model to xiaomi.keras
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 52ms/step - loss: 0.0204 - val_loss: 0.0027
Epoch 5/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.0091
Epoch 5: val_loss improved from 0.00274 to 0.00048, 

Root Mean Squared Error: 0.05535923801783168
Mean Absolute Error: 0.053767936508300654
Mean Squared Error: 0.0030646452339149403
R2 Score: 0.921734704444924


In [29]:
LSTM_model(
   normalized_train_nvidia,
   normalized_test_nvidia,
    'nvidia',
    scaler_nvidia
)

Epoch 1/50
30/32 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 0.0212
Epoch 1: val_loss improved from inf to 0.31786, saving model to nvidia.keras
32/32 ━━━━━━━━━━━━━━━━━━━━ 34s 242ms/step - loss: 0.0208 - val_loss: 0.3179
Epoch 2/50
29/32 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0091
Epoch 2: val_loss improved from 0.31786 to 0.23920, saving model to nvidia.keras
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 43ms/step - loss: 0.0089 - val_loss: 0.2392
Epoch 3/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0075
Epoch 3: val_loss improved from 0.23920 to 0.18599, saving model to nvidia.keras
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 52ms/step - loss: 0.0074 - val_loss: 0.1860
Epoch 4/50
30/32 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0057
Epoch 4: val_loss improved from 0.18599 to 0.04528, saving model to nvidia.keras
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 70ms/step - loss: 0.0056 - val_loss: 0.0453
Epoch 5/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.0017
Epoch 5: val_loss improved from 0.04528 to 0.01861, 

Root Mean Squared Error: 0.05424576587208771
Mean Absolute Error: 0.0503859812681647
Mean Squared Error: 0.0029426031150493553
R2 Score: 0.6795800782894119


In [27]:
LSTM_model(
   normalized_train_google,
   normalized_test_google,
    'google',
    scaler_google
)

Epoch 1/50
29/32 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.1973
Epoch 1: val_loss improved from inf to 0.43911, saving model to google.keras
32/32 ━━━━━━━━━━━━━━━━━━━━ 51s 197ms/step - loss: 0.1947 - val_loss: 0.4391
Epoch 2/50
29/32 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0961
Epoch 2: val_loss improved from 0.43911 to 0.04765, saving model to google.keras
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step - loss: 0.0923 - val_loss: 0.0476
Epoch 3/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.0249
Epoch 3: val_loss did not improve from 0.04765
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - loss: 0.0248 - val_loss: 0.0531
Epoch 4/50
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0185
Epoch 4: val_loss improved from 0.04765 to 0.02366, saving model to google.keras
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 69ms/step - loss: 0.0184 - val_loss: 0.0237
Epoch 5/50
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0137
Epoch 5: val_loss improved from 0.02366 to 0.00669, saving model to google.keras
32/32

Root Mean Squared Error: 0.05446340868584686
Mean Absolute Error: 0.05230547750297825
Mean Squared Error: 0.002966262885681579
R2 Score: 0.5810288429263838


In [28]:
LSTM_model(
   normalized_train_amazon,
   normalized_test_amazon,
    'amazon',
    scaler_amazon
)

Epoch 1/50
29/32 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.2796
Epoch 1: val_loss improved from inf to 0.47533, saving model to amazon.keras
32/32 ━━━━━━━━━━━━━━━━━━━━ 30s 142ms/step - loss: 0.2762 - val_loss: 0.4753
Epoch 2/50
30/32 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.1294
Epoch 2: val_loss improved from 0.47533 to 0.02564, saving model to amazon.keras
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - loss: 0.1257 - val_loss: 0.0256
Epoch 3/50
30/32 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0349
Epoch 3: val_loss did not improve from 0.02564
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 0.0346 - val_loss: 0.0383
Epoch 4/50
29/32 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0246
Epoch 4: val_loss improved from 0.02564 to 0.01283, saving model to amazon.keras
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.0244 - val_loss: 0.0128
Epoch 5/50
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0170
Epoch 5: val_loss improved from 0.01283 to 0.00727, saving model to amazon.keras
32/32

Root Mean Squared Error: 0.03587128938467412
Mean Absolute Error: 0.03334106053542248
Mean Squared Error: 0.0012867494021190343
R2 Score: 0.9426262800851302


In [31]:
print("Original Data:", train_apple['Close'].head())
print("Normalized Data:", normalized_train_apple[:5])


Original Data: 0    72.796028
1    72.088280
2    72.662720
3    72.320953
4    73.484344
Name: Close, dtype: float64
Normalized Data: [[0.11310729]
 [0.10872961]
 [0.11228273]
 [0.11016878]
 [0.11736478]]


In [32]:
print("Train shape:", normalized_train_apple.shape)
print("Test shape:", normalized_test_apple.shape)


Train shape: (1128, 1)
Test shape: (126, 1)


In [33]:
print("Scaler Min-Max Values:", scaler_apple.data_min_, scaler_apple.data_max_)


Scaler Min-Max Values: [54.509758] [216.18165588]
